In [1]:
!pip install wordfreq
from wordfreq import word_frequency
import spacy
!pip install openai
from openai import OpenAI
client = OpenAI(api_key='')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 31.0 MB/s eta 0:00:00
  Attempting uninstall: regex
    Found existing installation: regex 2023.6.3
    Uninstalling regex-2023.6.3:
      Successfully uninstalled regex-2023.6.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not 

In [2]:
def check_freq(sentence):

  nlp = spacy.load("en_core_web_sm")
  words = [tok.lemma_ for tok in nlp(sentence) if tok.pos_ not in ["PUNCT", "SPACE"]]

  freq_dict = {}
  for word in words:
    freq = word_frequency(word, 'en')
    freq_dict[word] = freq

  vocab = dict(sorted(freq_dict.items(), key=lambda item: item[1]))
  return vocab

In [3]:
def words_exp(sentence):

  freq_dict = check_freq(sentence)

  explain = dict((k, v) for k, v in freq_dict.items() if v < 1e-4)

  words = list(explain.keys())

  return words

In [4]:
def simplify_words(sentence):
  words = words_exp(sentence)
  assis = 'Give me a new verison of sentences which paraphrase these words in super easy-to-understand words:'
  for word in words:
    assis += word
    assis += ', '
  assis += 'inside sentences and only give me the new version of explained sentences.'

  completion = client.chat.completions.create(
      model='gpt-4-1106-preview',
      messages=[
         {"role": "system",
         "content": assis},
        {"role": "user",
         "content": sentence}
      ],
      temperature=0.5,
      top_p=0.5,
  )

  return completion.choices[0].message.content

In [5]:
def passive_transform(sentence):

  completion = client.chat.completions.create(
      model='gpt-4-1106-preview',
      messages=[
        {"role": "system",
         "content": "Change all passive sentences into active sentences."},
        {"role": "user",
         "content": sentence}
      ],
      temperature=0.5,
      top_p=0.5,
  )

  return completion.choices[0].message.content

In [6]:
def simplify_structure(sentence):
  completion = client.chat.completions.create(
      model='gpt-4-1106-preview',
      messages=[
        {"role": "system",
         "content": "Change all sentences into simple sentences without missing any important details and ensure the sentences are readable and coherent. You can break original sentences down to achieve this. Also do not increase words' complexity."},
        {"role": "user",
         "content": sentence}
      ],
      temperature=0.5,
      top_p=0.5,
  )
  return completion.choices[0].message.content

In [7]:
def simplify_phrases(sentence):
  completion = client.chat.completions.create(
      model='gpt-4-1106-preview',
      messages=[
        {"role": "system",
         "content": "Replace compound adjectives and challenging phrases with super simple synonyms."},
        {"role": "user",
         "content": sentence}
      ],
      temperature=0.5,
      top_p=0.5,
  )
  return completion.choices[0].message.content

In [8]:
def output(sentence):
    tmp = passive_transform(sentence)
    tmp2 = simplify_structure(tmp)
    tmp3 = simplify_phrases(tmp2)
    tmp4 = simplify_words(tmp3)
    tmp5 = simplify_structure(tmp4)
    output = simplify_words(tmp5)
    return output

In [9]:
output('If Dickens’s prose fiction has “defects”—excesses of melodrama, sentimentality, contrived plots, and manufactured happy endings—these are the defects of his era, which for all his greatness Dickens had not the rebellious spirit to resist; he was at heart a crowd-pleaser, a theatrical entertainer, with no interest in subverting the conventions of the novel as his great successors D.H. Lawrence, James Joyce, and Virginia Woolf would have; nor did he contemplate the subtle and ironic counterminings of human relations in the way of George Eliot and Thomas Hardy, who brought to the English novel an element of nuanced psychological realism not previously explored.')

"Dickens's stories are usually very exciting and full of feelings. They often have a happy ending. People liked this a lot when he was writing. Dickens aimed to make his readers have fun. He wasn't trying to make big changes in books. But later authors like D.H. Lawrence, James Joyce, and Virginia Woolf did change books. George Eliot and Thomas Hardy were not like Dickens. They used gentle jokes. They made their books feel more like real life. They let us see what the characters were thinking and feeling deep down."

In [ ]:
import pandas as pd
df = pd.read_csv('data.csv')
df = df.reset_index(drop=True)
original = df[df.columns[0]].tolist()
results = []
for statement in original:
    simplified = output(statement)
    results.append(simplified)

df_combined = pd.DataFrame({
    'OriginalStatements': original,
    'SimplifiedStatements': results,
})


df_combined.to_csv('simplification.csv', index=False)